In [95]:
import torch
import regex as re
from einops import rearrange, reduce, repeat
import os
import multiprocessing
from collections import defaultdict, Counter

In [148]:
# vocab init
encoding_vocab = {bytes([i]): i for i in range(256)}
vocab_max_position = 256
special_token = "<|endoftext|>"
encoding_vocab[special_token] = vocab_max_position
vocab_max_position += 1

In [137]:
#regex pattern to split text
PAT = r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [149]:
test_corpus = '''low low low low low
lower lower widest widest widest
newest newest newest newest newest newest'''


In [139]:
# counting_dict: dict[tuple[bytes], int] = defaultdict(int)
# for match in re.finditer(PAT, test_corpus):
#     counting_dict[match.group()] += 1

In [185]:
parts = re.findall(r"\S+", test_corpus)
parts = [tuple(i) for i in parts]
counting_dict = defaultdict(int, Counter(parts)) #Counter is a cool pythonic way to make a dict
parts = list(set(parts)) #making parts unique after counting is done
merges = []

In [ ]:
merge_dict = defaultdict(int)

for i,j in counting_dict.items():
    total_letters = len(i)
    k = 0
    while (k+1) < total_letters:
        merge_dict[i[k] + i[k+1]] += 1 * j
        k +=1

#lexiographically higherst occuring one:
new_token = max(merge_dict.items(), key=lambda x: (x[1], x[0]))[0]

encoding_vocab[new_token] = vocab_max_position
vocab_max_position += 1
        

to_be_removed = []
for i in parts:
    lenght = len(i)
    j = 0
    while (j+1) < lenght:
        if (i[j] + i[j+1]) == new_token:
            to_be_removed.append(i)
            new_i = i[:j] + (new_token,) + i[j+2:]
            counting_dict[new_i] = counting_dict.pop(i)
            parts.append(new_i)
        j+= 1


    
for i in to_be_removed:
    parts.remove(i)



In [187]:
print(merges)

[('l', 'o'), ('o', 'w'), ('l', 'o'), ('o', 'w'), ('w', 'e'), ('e', 'r'), ('w', 'i'), ('i', 'd'), ('d', 'e'), ('e', 's'), ('s', 't'), ('n', 'e'), ('e', 'w'), ('w', 'e'), ('e', 's'), ('s', 't')]


In [166]:
print(encoding_vocab)

{b'\x00': 0, b'\x01': 1, b'\x02': 2, b'\x03': 3, b'\x04': 4, b'\x05': 5, b'\x06': 6, b'\x07': 7, b'\x08': 8, b'\t': 9, b'\n': 10, b'\x0b': 11, b'\x0c': 12, b'\r': 13, b'\x0e': 14, b'\x0f': 15, b'\x10': 16, b'\x11': 17, b'\x12': 18, b'\x13': 19, b'\x14': 20, b'\x15': 21, b'\x16': 22, b'\x17': 23, b'\x18': 24, b'\x19': 25, b'\x1a': 26, b'\x1b': 27, b'\x1c': 28, b'\x1d': 29, b'\x1e': 30, b'\x1f': 31, b' ': 32, b'!': 33, b'"': 34, b'#': 35, b'$': 36, b'%': 37, b'&': 38, b"'": 39, b'(': 40, b')': 41, b'*': 42, b'+': 43, b',': 44, b'-': 45, b'.': 46, b'/': 47, b'0': 48, b'1': 49, b'2': 50, b'3': 51, b'4': 52, b'5': 53, b'6': 54, b'7': 55, b'8': 56, b'9': 57, b':': 58, b';': 59, b'<': 60, b'=': 61, b'>': 62, b'?': 63, b'@': 64, b'A': 65, b'B': 66, b'C': 67, b'D': 68, b'E': 69, b'F': 70, b'G': 71, b'H': 72, b'I': 73, b'J': 74, b'K': 75, b'L': 76, b'M': 77, b'N': 78, b'O': 79, b'P': 80, b'Q': 81, b'R': 82, b'S': 83, b'T': 84, b'U': 85, b'V': 86, b'W': 87, b'X': 88, b'Y': 89, b'Z': 90, b'[': 91,

In [ ]:

encoding_vocab

{b'\x00': 0,
 b'\x01': 1,
 b'\x02': 2,
 b'\x03': 3,
 b'\x04': 4,
 b'\x05': 5,
 b'\x06': 6,
 b'\x07': 7,
 b'\x08': 8,
 b'\t': 9,
 b'\n': 10,
 b'\x0b': 11,
 b'\x0c': 12,
 b'\r': 13,
 b'\x0e': 14,
 b'\x0f': 15,
 b'\x10': 16,
 b'\x11': 17,
 b'\x12': 18,
 b'\x13': 19,
 b'\x14': 20,
 b'\x15': 21,
 b'\x16': 22,
 b'\x17': 23,
 b'\x18': 24,
 b'\x19': 25,
 b'\x1a': 26,
 b'\x1b': 27,
 b'\x1c': 28,
 b'\x1d': 29,
 b'\x1e': 30,
 b'\x1f': 31,
 b' ': 32,
 b'!': 33,
 b'"': 34,
 b'#': 35,
 b'$': 36,
 b'%': 37,
 b'&': 38,
 b"'": 39,
 b'(': 40,
 b')': 41,
 b'*': 42,
 b'+': 43,
 b',': 44,
 b'-': 45,
 b'.': 46,
 b'/': 47,
 b'0': 48,
 b'1': 49,
 b'2': 50,
 b'3': 51,
 b'4': 52,
 b'5': 53,
 b'6': 54,
 b'7': 55,
 b'8': 56,
 b'9': 57,
 b':': 58,
 b';': 59,
 b'<': 60,
 b'=': 61,
 b'>': 62,
 b'?': 63,
 b'@': 64,
 b'A': 65,
 b'B': 66,
 b'C': 67,
 b'D': 68,
 b'E': 69,
 b'F': 70,
 b'G': 71,
 b'H': 72,
 b'I': 73,
 b'J': 74,
 b'K': 75,
 b'L': 76,
 b'M': 77,
 b'N': 78,
 b'O': 79,
 b'P': 80,
 b'Q': 81,
 b'R': 82,
 b'S': 

In [143]:
counting_dict

defaultdict(int,
            {('l', 'o', 'w'): 5,
             ('l', 'o', 'w', 'e', 'r'): 2,
             ('w', 'i', 'd', 'e', 's', 't'): 3,
             ('n', 'e', 'w', 'e', 's', 't'): 6})

[('w', 'i', 'd', 'e', 's', 't')]
[('w', 'i', 'd', 'e', 's', 't'), ('n', 'e', 'w', 'e', 's', 't')]


In [145]:
counting_dict

defaultdict(int,
            {('l', 'o', 'w'): 5,
             ('l', 'o', 'w', 'e', 'r'): 2,
             ('w', 'i', 'd', 'e', 'st'): 3,
             ('n', 'e', 'w', 'e', 'st'): 6})

In [146]:
parts

[('l', 'o', 'w', 'e', 'r'),
 ('l', 'o', 'w'),
 ('w', 'i', 'd', 'e', 'st'),
 ('n', 'e', 'w', 'e', 'st')]

In [147]:
#ok, now I just gotta wrap all of this shit in a function, so I can make sure that this code is reusable and can be run many many times.

In [ ]:
# initial vocab:
encoding_vocab = {bytes([i]): i for i in range(256)}
special_token = "<|endoftext|>"
encoding_vocab[bytes(special_token)] = 256

In [ ]:
re.finditer()

In [31]:
fd = os.open("../../data/TinyStoriesV2-GPT4-valid.txt", os.O_RDONLY)
data = os.read(fd, os.path.getsize("../../data/TinyStoriesV2-GPT4-valid.txt"))
os.close(fd)
text = data.decode("utf-8")

In [38]:

# 1) your PAT (Unicode-aware)
PAT = r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
pat = re.compile(PAT)

# special tokens to strip/split on
specials = ["<|endoftext|>"]  # add more if needed
splitter = re.compile("|".join(re.escape(t) for t in specials))

def pretokenize(text):
    tokens = []
    for chunk in splitter.split(text):      # 1) split across specials
        if not chunk:                       # 2) specials are stripped (not captured)
            continue
        for m in pat.finditer(chunk):       # 3) run PAT per chunk
            tokens.append(m.group())
    return tokens


In [40]:
len(pretokenize(text))

5419001

In [ ]:
def run_train_bpe(
    input_path: str | os.PathLike,
    vocab_size: int,
    special_tokens: list[str],
    **kwargs,
) -> tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
    """Given the path to an input corpus, run train a BPE tokenizer and
    output its vocabulary and merges.

    Args:
        input_path (str | os.PathLike): Path to BPE tokenizer training data.
        vocab_size (int): Total number of items in the tokenizer's vocabulary (including special tokens).
        special_tokens (list[str]): A list of string special tokens to be added to the tokenizer vocabulary.
            These strings will never be split into multiple tokens, and will always be
            kept as a single token. If these special tokens occur in the `input_path`,
            they are treated as any other string.

    Returns:
        tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
            vocab:
                The trained tokenizer vocabulary, a mapping from int (token ID in the vocabulary)
                to bytes (token bytes)
            merges:
                BPE merges. Each list item is a tuple of bytes (<token1>, <token2>),
                representing that <token1> was merged with <token2>.
                Merges are ordered by order of creation.
    """
    raise NotImplementedError

- how does multiprocessing work?
